In [20]:
import requests
import io
import json
from pprint import pprint
from requests.exceptions import HTTPError,ConnectionError,Timeout

try:
    url = 'https://opendata.cwa.gov.tw/fileapi/v1/opendataapi/O-A0091-001?Authorization=rdec-key-123-45678-011121314&format=JSON'
    response = requests.get(url)
    response.raise_for_status()
    #print(type(response))
    content = response.text
    #print(content)
except ConnectionError:
    print("連線伺服器發生錯誤")
except Timeout:
    print("伺服器太忙,沒有回應")
except HTTPError:
    print("伺服器回應連線錯誤")
except Exception:
    print("不知名的錯誤")

file=io.StringIO(content)
content=json.load(file)
date:str=content['cwaopendata']['sent']
stations=content['cwaopendata']['dataset']['Station']

class Station():
    def __init__(self,name:str,weather:float):
        self.name=name
        self.weather=weather

stations1:list=[]
for station in stations:
    name=station['StationName']
    weather=station['WeatherElement']['SolarRadiation']
    station:Station=Station(name,float(weather))
    stations1.append(station)

stations1